# Zadanie 3
### Interactions
Załóżmy, że jeśli Ania i Beata skomentowały ten sam film movie_1, dochodzi między nimi do bezpośredniej interakcji. Możemy sobie wyobrazić, że w pewien sposób rozmawiali ze sobą.
Jeśli Beata i Czarek wypowiedzieli się pod filmem movie_2, między nimi doszło również do interakcji.
W końcu Beata mogła wspomnieć w komentarzu pod filmem movie_2 Czarkowi o tym, czego dowiedziała się z komentarza Ani pod filmem movie_1. W ten sposób przyjmujemy, że pomiędzy Anią i Czarkiem pośrednio również doszło do interakcji.
Jeśli podzielimy w ten sposób wszystkich użytkowników na rozłączne grupy otrzymamy poszukiwane grupy interakcji. Co ważne – Ania będzie też w interakcji z każdą osobą, która pośrednio lub bezpośrednio jest w interakcji z Czarkiem.

#### Twoje zadanie polega na odpowiedzeniu ile mamy takich rozłącznych grup, jeśli:

o dataset
- nie ma dla nas znaczenia, czy user skomentował filmy należące do tej samej kategorii czy dotyczące tej samej gry. Istotne jest jedynie to, że doszło do interakcji w pewnych filmach.

o Group_game

- dzielimy użytkowników na grupy wyłącznie w oparciu o ich aktywność pod filmami wybranymi dla gier w tej kategorii. Tym samym Ania komentując filmy w dwóch różnych kategoriach będzie występowała w dwóch grupach. Liczba, której szukamy to faktycznie suma liczb rozłącznych grup interakcji dla wszystkich kategorii gier.

o Game
- posługując się tą samą logiką, co w przypadku group_game, szukamy sumy liczb rozłącznych grup interakcji dla wszystkich gier.

In [1]:
# Imports
import pandas as pd
#import numpy as np
from pandasql import sqldf

# SQL query function
def pysqldf(q):
    return sqldf(q, globals())
#    pysqldf = lambda q: sqldf(q, globals())

In [2]:
# folders, dataframes
data_path = 'data/' # data folder
data_frames  = ["comments","game_groups","game_movies","games"] #data csc files
output_folder = 'output/'# output folder

In [3]:
# read "zadania"
group_scores_Tomasz_Cwik = pd.read_csv("group_scores_name_surname.csv",encoding = 'utf-8',sep = ',')
game_scores_Tomasz_Cwik = pd.read_csv("game_scores_name_surname.csv",encoding = 'utf-8',sep = ',')
interactions_Tomasz_Cwik = pd.read_csv("interactions_name_surname.csv",encoding = 'utf-8',sep = ',')

#group_scores_Tomasz_Cwik 

In [4]:
# read tables
comments = pd.read_csv(data_path + "comments.csv", encoding = 'utf-8',sep = ',')
game_groups = pd.read_csv(data_path + "game_groups.csv", encoding = 'utf-8',sep = ',')
game_movies = pd.read_csv(data_path + "game_movies.csv",encoding = 'utf-8',sep = ',')
games = pd.read_csv(data_path + "games.csv",encoding = 'utf-8',sep = ',')

### Extra tables or variables

In [5]:
# create extra tables or variables

#list ot tabals or lists
# ----- distinct users ------df-------

# ---------- distinct users from comments

q = """
select distinct user from comments
"""
duser = pysqldf(q)

# ------- duser_number ---int------
duser_number =  duser['user'].count()
print("Total distinct useres in comments table: ",duser_number)
#duser # distinct users


# ---------- distinct movies Nodes from comments



Total distinct useres in comments table:  75429


# 3.1 Dataset
- nie ma dla nas znaczenia, czy user skomentował filmy należące do tej samej kategorii czy dotyczące tej samej gry. Istotne jest jedynie to, że doszło do interakcji w pewnych filmach.


Logiga poszukiwania:

- Definiuję węzeł "node" jako film (id_movie) z tabeli comments
- Szukamy interakcji zaczynając od dowolnego Noda (może to myć np. min(id_movie))
- Szukamy innych nodów (id_movie), które weszły w interakcję
- znalezione nody dodajemy do grupy znalezionych  - "zainfekowanych"
- zawężamy obszar poszukiwań o już przypisane nody
- powtarzamy interację do momentu przeszukania i przypisania wszystkich nodów

In [6]:
# interactions dataframe - df to store interactions groups
#interactions_dataset = pd.DataFrame(columns = ['id_node','id_inter_group','id_movie', 'user')


q = """
select distinct id_movie from comments
"""
d_movies_list = pysqldf(q)['id_movie'].tolist()
#d_movies_list


all_nodes_list = d_movies_list 

# just helpfull printouts
print ("Node is an id_movie")
print("All analysed nodes: ", len(all_nodes_list))
print ("list of nodes to search:")
print (all_nodes_list)

Node is an id_movie
All analysed nodes:  144
list of nodes to search:
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 61, 62, 63, 64, 65, 66, 67, 69, 70, 74, 76, 77, 78, 79, 82, 83, 84, 85, 86, 87, 89, 91, 101, 103, 105, 108, 110, 111, 113, 115, 117, 118, 119, 126, 130, 137, 140, 142, 146, 147, 162, 163, 164, 166, 167, 168, 169, 170, 171, 172, 173, 174, 177, 178, 180, 183, 186, 187, 191, 193, 194, 196, 197, 198, 200, 202, 203, 205, 206, 208, 211, 215, 216, 218, 219, 221, 224, 225, 232, 238, 239, 241, 243, 244, 249, 251, 252, 254]


In [7]:
# Main cell

# interactions dataset to stora data of interactions
interactions_dataset = pd.DataFrame(columns = ['id_interaction','id_node','type']) # here we stora info of interactions
interaction_groups_count = 0 # cunt the numner of groups search and found
infected_nodes = [] # define a list of nodes we found, lets start with empty
end_task_search = False # loop end variable
end_task_search_count = 0 # Groups counter
remain_nodes =  all_nodes_list # Definition of dataset to search - lets make put all data at start

print ("All groups in dataset: ", len(all_nodes_list))
print (all_nodes_list)

while end_task_search == False: # loop searchin the groups of interactions
   
    end_task_search_count +=1 
    
    # define nodes to search - name "search_node"
    interaction_groups_count +=1
    
    print ("-"*50)
    print ("")
    print (">> Searching group #:  ", interaction_groups_count)
    print ("")
    print ("-"*50)
    
    
    remain_nodes = [x for x in remain_nodes if x not in infected_nodes]
    infected_nodes = [] # zerowanie infected notes
    
    
    if remain_nodes: # testing if we should end the process
        print ("still searchig...")
    else:
        print ("end of search")
        end_task_search = False
        break
        
    print ("-"*45)
    print ("Number of group of interactions to search: ", interaction_groups_count)
    print ("Nodes infected",infected_nodes)
    print ("Ramaing nodes to check ",remain_nodes)
    print ("-"*45)

    search_node = "(" + str(remain_nodes[0]) + ")"
    infected_nodes = [remain_nodes[0]] # definiuje pierwszego szukanego jako zarażonego 
    
    #test if we switch the order
    #if interaction_groups_count == 1:
    #    search_node = "(" + str(remain_nodes[122]) + ")"
    #    infected_nodes = [remain_nodes[122]]
    
    #print ("Search nodes: ",search_node)
    

    infected_nodes_sql = search_node
    #print (search_node)
    # Loop for seraching interactions (start from no 1)
    # zeros of 
    i=0
    end_graph_search = False
    
    
    # -- Loop for seraching interactions (start from no 1)----------------------------------------------------------------------------------------------------------------
    while end_graph_search == False:
        i= i + 1
        print ("---------------------")
        print ("Interation #: ", i)
        print ("Seach nodes: " )
        print (search_node)


        # step 1

        # step 1.1 Search all users that could have been infected with Node
        # That case is id_movie
        q_step_1 = """
            select distinct user
            from comments
            where id_movie in """ + str(search_node) + """

            """
        infected_users = pysqldf(q_step_1)

        #Step 2
        # Search of nodes that are infected with useres from the previous node 
    
        q= """
            with next_infected_nodes_search as (

                select distinct id_movie 
                from comments 
                    where
                    user in (

                            select distinct user
                            from comments
                            where 
                                id_movie in """ + str(search_node) + """



                            )   
                                            )
            select id_movie from next_infected_nodes_search
                where id_movie NOT IN """ + infected_nodes_sql + """ /* bez już znalezionych */
            """                     


        #print(q)
        next_infected_nodes = pysqldf(q)

        # NEXT INFECTED
        # ----------------------------------------------------------------------
        #next_infected_nodes found list
        next_infected_nodes_list = next_infected_nodes['id_movie'].tolist()

        # lests make string for next iteration for SQL select
        search_node =""
        for x in next_infected_nodes_list:
            search_node += str(x)+","
        search_node = "(" + search_node[:-1] + ")"
        #print ("Tomek search node", search_node) # intermidiate test
        
        # lets count how many was found
        next_infected_nodes_list_count = len(next_infected_nodes_list)
        
        # Add infected nodes 
        infected_nodes = infected_nodes +  next_infected_nodes_list 
        #print ("infected_nodes + test next_infected_nodes_list ",infected_nodes) # Test print

        # lets make string for SQL of infected_nodes
        infected_nodes_sql =""
        for x in infected_nodes:
            infected_nodes_sql += str(x)+","
        infected_nodes_sql = "(" + infected_nodes_sql[:-1] + ")"
        
                
        # If to check if we should stop the searchin becasue no more nodes are found
        if next_infected_nodes_list_count == 0:
            # nothing was found we stop to loop
            print ()
            print ("Did not find any relationship /any infected nodes/...")
            print ("End of iterations")
            print ("----------------------- End Result -----------------------------------")
            print ("Infected nodes found: " , len(infected_nodes) )
            print ("List of found: ")
            print (infected_nodes)

            print ("Id of group of interactions: " , end_task_search_count)
            print ("----------------------------------------------------------------------")
            
            end_graph_search = True
            
        else:
            # There are still nodes to search
            print()
            print ("Infected nodes found: ", next_infected_nodes_list_count)
            print ()
            print ("List of found and infected: ")
            print (next_infected_nodes_list)
            print ()
            print ("Total nodes found: " , len(infected_nodes) )
            print (infected_nodes)
            print ()

        # end of the loop


    # Building a datafreame with info
    f=0
    #print ("test len range", len(infected_nodes))
    #print ("test len range od 0 ", infected_nodes[0])
    for f in range(len(infected_nodes)):
        # Append Row to DataFrame
        #print ("f: ",f, " i: ", i)
        
        list_row = [end_task_search_count, infected_nodes[f], "dataset"]
        interactions_dataset.loc[len(interactions_dataset)] = list_row
    
    
    interactions_dataset



All groups in dataset:  144
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 61, 62, 63, 64, 65, 66, 67, 69, 70, 74, 76, 77, 78, 79, 82, 83, 84, 85, 86, 87, 89, 91, 101, 103, 105, 108, 110, 111, 113, 115, 117, 118, 119, 126, 130, 137, 140, 142, 146, 147, 162, 163, 164, 166, 167, 168, 169, 170, 171, 172, 173, 174, 177, 178, 180, 183, 186, 187, 191, 193, 194, 196, 197, 198, 200, 202, 203, 205, 206, 208, 211, 215, 216, 218, 219, 221, 224, 225, 232, 238, 239, 241, 243, 244, 249, 251, 252, 254]
--------------------------------------------------

>> Searching group #:   1

--------------------------------------------------
still searchig...
---------------------------------------------
Number of group of interactions to search:  1
Nodes infected []
Ramaing nodes to check  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 

In [8]:
interactions_dataset

,id_interaction,id_node,type
0,1,1,dataset
1,1,3,dataset
2,1,4,dataset
3,1,5,dataset
4,1,6,dataset
...,...,...,...
139,13,119,dataset
140,14,130,dataset
141,15,137,dataset
142,16,191,dataset


In [9]:
#infected_nodes
#nodes_dataset = pd.DataFrame(infected_nodes , columns= ['id_movie'])
interactions_dataset

interactions_dataset.to_csv(path_or_buf  = "Zadanie_3_1_interaction_users_groups_Tomasz_Cwik.csv", index = False, encoding = 'utf-8' )
#del nodes_dataset

In [10]:
# Making and saving list of groups with users

q = """

Select i.id_node, c.id_movie, c.user
    from interactions_dataset as i
        left join (
                    select distinct id_movie, user from comments
                     ) as c
        
        on i.id_node = c.id_movie
        

"""
interaction_users = pysqldf(q)
interaction_users
interaction_users.to_csv(path_or_buf  = "Zadanie_3_1_interaction_users_groups_Tomasz_Cwik_useres.csv", index = False, encoding = 'utf-8' )